In [ ]:
import feedparser

from bs4 import BeautifulSoup

from whoosh.index import create_in
from whoosh.fields import *
from whoosh.qparser import QueryParser, FuzzyTermPlugin
from whoosh import highlight

import calendar
from datetime import datetime

In [ ]:
import feedparser

NewsFeed = feedparser.parse("https://thegeneralist.substack.com/feed")

entry = NewsFeed.entries[0]
print(entry)

In [ ]:
print(entry['title'])
print(entry['author'])
print(entry['summary'])
print(entry['link'])
print(entry['published_parsed'])
content = entry['content'][0]['value']
print(content)

In [ ]:
# Turn HTML into clean text for nice searching

from bs4 import BeautifulSoup
tree = BeautifulSoup(content)

# This validates and cleans HTML but Substack HTML is already fine
# pretty = tree.prettify()
# print(pretty)

pure_text = tree.get_text("\n")
print(pure_text)

In [ ]:
# Next, build a Whoosh database and search engine
# https://whoosh.readthedocs.io/en/latest/quickstart.html#a-quick-introduction

# Define schema (pretty simple)
# Add documents from all top Substacks
# Build a searcher
# Show results
# Highlight bits from the thinkpieces that match
# Optional: add stemming for better searching
# Later, add support for other newsletters with an RSS feed
# Maybe, later, offer `more_like_this` so people can rabbit-hole in

# Use case: I want to find all articles from my favorite writers that include the term
# "BNPL" since I want to research "Buy Now, Pay Later" in fintech.
# Or I want to find all thinkpieces that mentioned Google in the last year

In [ ]:
# Create a list of all RSSes to search

# First, Substacks. Getting their RSS feeds is pretty simple/systematic.
substack_domains = [
    "thegeneralist",
    "danco",
    "diff",
    "nbt",
    "platformer",
    "notboring",
    "sariazout",
    "digitalnative",
    "jamesonstartups",
    "breakingsmart",
    "artofgig",
    "theskip",
    "gwern"
]

# Feeds are, e.g., https://thegeneralist.substack.com/feed
substack_feeds = ["https://{0}.substack.com/feed".format(domain) for domain in substack_domains]

# Now add some custom RSS feeds
# Medium feeds are medium.com/feed/@user or medium.com/feed/publication
custom_feeds = [
    "https://stratechery.com/feed/",
    "https://www.profgalloway.com/feed",
    "https://eugene-wei.squarespace.com/blog?format=rss",
    "https://medium.com/feed/@superwuster",
    "https://commoncog.com/blog/rss",
    "https://www.lennyrachitsky.com/feed",
    "https://medium.com/feed/bloated-mvp",
    "https://daringfireball.net/feeds/main",
    "https://wongmjane.com/api/feed/rss",
    "https://fourweekmba.com/feed",
]

# Unite all feeds into one
all_feeds = substack_feeds + custom_feeds
all_feeds

# We'll read in the feeds from each of these. Get RSS feed at 
# https://thegeneralist.substack.com/feed

# NewsFeed = feedparser.parse("https://thegeneralist.substack.com/feed")

In [ ]:
# Convenience function to safely get an item from a dict.
# If the key doesn't exist, just returns none
def safe_get(obj, key):
    if obj.has_key(key):
        return obj[key]
    else:
        return None

In [ ]:
# Allows optional case-sensitive searches. all-lowercase is case insensitive, 
# any capital letters makes it case sensitive 

class CaseSensitivizer(analysis.Filter):
    def __call__(self, tokens):
        for t in tokens:
            yield t
            if t.mode == "index":
               low = t.text.lower()
               if low != t.text:
                   t.text = low
                   yield t

ana = analysis.RegexTokenizer() | CaseSensitivizer()
# [t.text for t in ana("The new SuperTurbo 5000", mode="index")]
# ["The", "the", "new", "SuperTurbo", "superturbo", "5000"]



In [ ]:
# Build the database
schema = Schema(
    title=TEXT(stored=True),
    author=TEXT(stored=True),
    publication=TEXT(stored=True),
    summary=TEXT(stored=True),
    url=TEXT(stored=True),
    published=DATETIME(stored=True, sortable=True),
    content=TEXT(stored=True, analyzer=ana))
index = create_in("whoosh_index2", schema)
writer = index.writer()

# Read every item from our RSS feeds into there
# Call writer.add_document() repeatedly for each item 

for feed_url in all_feeds:
    print(feed_url)
    news_feed = feedparser.parse(feed_url)
    
    # NOTE: we can only get the last few entries from this RSS feed.
    # Substack doesn't seem to show anything older than the last 20.
    # So we should build in a system to start caching these.
    
#     print(len(news_feed.entries))

    for entry in news_feed.entries:
        
        # Get publication name. This is in the feed's `feed` field, along with other metadata
        publication = None
        metadata = safe_get(news_feed, 'feed')
        if metadata is not None:
            publication = safe_get(metadata, 'title')

        # Clean up the date into a normal datetime
        clean_datetime = datetime.fromtimestamp(calendar.timegm(entry['published_parsed']))
        
        # Most feeds put the main content in `content`,
        # but a rare few like Eugene Wei put it in `summary`
        # (in which case `content` is empty). With this logic, let's get a single `content` field.
        body_text = None
        # See if `content` exists
        content_holder = safe_get(entry, 'content')
        if content_holder is not None:
            # We have content; fill it in
            content_tree = BeautifulSoup(content_holder[0]['value'])
            body_text = content_tree.get_text(" ", strip=True)
        else:
            # No content provided. `summary` must hold all the text.
            summary_tree = BeautifulSoup(safe_get(entry, 'summary'))
            body_text = summary_tree.get_text(" ", strip=True)

        writer.add_document(
            title=safe_get(entry, 'title'),
            author=safe_get(entry, 'author'),
            publication=publication,
            summary=safe_get(entry, 'summary'),
            url=safe_get(entry, 'link'),
            published=clean_datetime,
            content=body_text)

print("DONE!")
writer.commit()

In [ ]:
# For convenience, we're overriding the standard fragment formatter
class BracketFormatter(highlight.Formatter):
    """Puts square brackets around the matched terms.
    """

    def format_token(self, text, token, replace=False):
        # Use the get_text function to get the text corresponding to the
        # token
        tokentext = highlight.get_text(text, token, replace)

        # Return the text as you want it to appear in the highlighted
        # string
        return "[[%s]]" % tokentext

In [ ]:
# Try searching
from whoosh.qparser import QueryParser, MultifieldParser
import whoosh.qparser as qparser

search_term = "Notion"

with index.searcher() as searcher:
    parser = QueryParser("content", index.schema)
    # Allow fuzzy matching (EDIT: kinda screws things up)
    # parser.add_plugin(FuzzyTermPlugin())
    # Allow searching for entire phrases w/ single quotes, like 'microsoft teams'
    parser.add_plugin(qparser.SingleQuotePlugin())
    
    query = parser.parse(search_term)
    results = searcher.search(query, limit=None)
    
    # Highlighting settings
    # This provides more context characters around the searched-for text
    results.fragmenter.surround = 50
    results.fragmenter.maxchars = 500
    
    # Surround matched tags with brackets
    results.formatter = BracketFormatter()
    
    # Convert each Hit into a dict
    def extract_hit_info(hit):
        return {
            'title': hit.get('title'),
            'publication': hit.get('publication'),
            'author': hit.get('author'),
            'url': hit.get('url'),
            'highlights': hit.highlights("content", top=3),
            'published': hit.get('published'),
            'score': hit.score
        }
    
    hit_list = [extract_hit_info(h) for h in results]
    
    print(hit_list)

In [ ]:
# Experimental

news_feed = feedparser.parse("https://diff.substack.com/feed")
print(news_feed['feed'])

In [ ]:
# TODO: read from the thinkpiecer module and use it here, instead of writing custom code

In [ ]:
z = {'a':5, 'b':3}

In [ ]:
print(z.get('c'))

In [ ]:
# Search for the newest items.

# Try searching
from whoosh.qparser import QueryParser, MultifieldParser
import whoosh.qparser as qparser
from whoosh.qparser.dateparse import DateParserPlugin

search_term = "'-26 weeks to now'"

with index.searcher() as searcher:
    parser = QueryParser("published", index.schema)
    # Allow fuzzy matching (EDIT: kinda screws things up)
    # parser.add_plugin(FuzzyTermPlugin())
    # Allow searching for entire phrases w/ single quotes, like 'microsoft teams'
    parser.add_plugin(qparser.SingleQuotePlugin())
    
    # Add the DateParserPlugin to the parser
    parser.add_plugin(DateParserPlugin())
    
    query = parser.parse(search_term)
    results = searcher.search(query, limit=50, sortedby="published", reverse=True)
    
    # Highlighting settings
    # This provides more context characters around the searched-for text
#     results.fragmenter.surround = 50
#     results.fragmenter.maxchars = 500
    
    # Surround matched tags with brackets
#     results.formatter = BracketFormatter()
    
    # Convert each Hit into a dict
    def extract_hit_info(hit):
        return {
            'title': hit.get('title'),
            'publication': hit.get('publication'),
            'author': hit.get('author'),
            'url': hit.get('url'),
#             'highlights': hit.highlights("content", top=3),
            'published': hit.get('published'),
            'score': hit.score
        }
    
    hit_list = [extract_hit_info(h) for h in results]
    
    print(hit_list)

In [1]:
from thinkpiecer import *

# Let's try the real code
# Load the real index and search it 

ix = load_index()

# Figure out the distribution of word counts. My theory is that there are a lot of
# low-quality, "preview-only / paywalled" pieces contained here and they should be filtered out.
# Is there a breakpoint from bad to good?
recs = get_recent_articles(ix)
print(recs)

[{'title': 'EHR interoperability: why it matters', 'publication': 'In Silico', 'author': 'Scott Xiao', 'url': 'https://insilico.substack.com/p/ehr-interoperability-why-it-matters', 'published': datetime.datetime(2020, 12, 14, 17, 44, 23), 'score': 63743564663000000}, {'title': 'Everybody Hates Facebook', 'publication': 'Not Boring by Packy McCormick', 'author': 'Packy McCormick', 'url': 'https://notboring.substack.com/p/everybody-hates-facecbook', 'published': datetime.datetime(2020, 12, 14, 9, 7, 21), 'score': 63743533641000000}, {'title': 'Everybody Hates Facebook', 'publication': 'Not Boring by Packy McCormick', 'author': 'Packy McCormick', 'url': 'https://notboring.substack.com/p/everybody-hates-facebook', 'published': datetime.datetime(2020, 12, 14, 8, 54, 46), 'score': 63743532886000000}, {'title': 'What comes after smartphones?', 'publication': 'Essays - Benedict Evans', 'author': 'Benedict Evans', 'url': 'https://www.ben-evans.com/benedictevans/2020/12/13/what-comes-after-smart

In [ ]:
import calendar
calendar.timegm

In [3]:
import thinkpiecer
import feedparser
from bs4 import BeautifulSoup
from whoosh import index
from whoosh.fields import *
from whoosh.writing import AsyncWriter
from whoosh.qparser import QueryParser
import whoosh.qparser as qparser
from whoosh.qparser.dateparse import DateParserPlugin
from whoosh import highlight
import calendar
from datetime import datetime
import os, os.path

# Local modules
import utilities
from utilities import safe_get
import feeds


In [22]:
# Try searching for recent high quality articles
ix = thinkpiecer.load_index()

with ix.searcher() as searcher:
    # Search for the newest items.
    # We have to search for SOMETHING so let's filter out all the
    # articles that are too short. Then we can sort by date desc.
    parser = QueryParser("content_word_count", ix.schema)
    
    # Add a parser to do numerical comparisons
    # This is the "greater than, less than" plugin
    parser.add_plugin(qparser.GtLtPlugin())
    search_term = "content_word_count:>=250"
    query = parser.parse(search_term)
    

    # We have to search for SOMETHING
    # so we'll start by just searching for recent articles.
#     parser = QueryParser("published", ix.schema)
#     parser.add_plugin(DateParserPlugin())
    # Let's limit the amount of sorting we have to do by only looking
    # at pieces in recent history
#     search_term = "'-26 weeks to now'"
#     query = parser.parse(search_term)
    results = searcher.search(query,
        limit=10, sortedby="published", reverse=True)

    # Convert each Hit into a dict
    def extract_hit_info(hit):
        return {
            'title': hit.get('title'),
            'publication': hit.get('publication'),
            'author': hit.get('author'),
            'url': hit.get('url'),
            'published': hit.get('published'),
            'content_word_count': hit.get('content_word_count'),
            'description': hit.get('description'),
            'score': hit.score
        }

    hit_list = [extract_hit_info(h) for h in results]
    print([h['description'] for h in hit_list])

['New HHS rules set the stage for consumer EHRs and making health data portable', "Listen now (32 min) | Welcome to the 2,121 newly Not Boring people who have joined us since last Monday! If you aren’t subscribed, join 25,452 smart, curious folks by subscribing here! 🎧 To get this essay straight in your ears: listen on Spotify. This week's Not Boring is brought to you by…", 'Understanding the Ambitions of the Company We All Love to Hate', 'Computing has always moved forwards in jumps of scale, but smartphones \nreach everyone on earth, so what’s the next jump? AV and VR? Cloud, machine \nlearning and crypto? Or is that the wrong model to use?', 'What are cancer care navigators and what issues do they face?', 'Welcome to new subscribers. The Generalist helps you master the future by analyzing the most significant tech and financial market trends. Thank you for being here. There are now over 19,000 of us! Hey friends, The IPO frenzy reached its zenith with DoorDash and Airbnb making thei

In [11]:
z = "Putting it All Together. Facebook is an advertising juggernaut on pace to do more than $80 billion in revenue this year at 80+% gross margins, which has only become more critical for advertisers during the pandemic. Because it’s hated, you get all of that at a slight discount without accounting for Facebook’s wild bets on the future. Just to be extra clear: this is not a value judgement. Facebook may very well be evil. As I was writing this post, I hopped on to Twitter, and this is the first tweet that popped into my feed: But because we all think Facebook is evil, we don’t spend much time trying to understand the full scope of the business behind the products on which we spend 65 minutes per day. What is Facebook? Facebook is so omnipresent that the analysis of the company is all trees, no forest. When I started researching this piece, I realized that despite reading Stratechery daily and spending a lot of time on tech twitter, I don’t fully understand everything the company does and how it all fits together. So let’s all admit that we’ve lost track, hit reset, zoom out, and admire the forest. Nearly half of the world’s population uses one or more of Facebook’s four main social media properties - Facebook , Messenger , Instagram , and WhatsApp . Facebook is also building what it hopes is the next computing platform through Facebook Reality Labs, which houses Oculus, Portal, Spark AR, CTRL-labs, and more. The main Facebook product had 1.8 billion Daily Active Users (DAUs) and 2.7 billion Monthly Active Users"
z[:250]

'Putting it All Together. Facebook is an advertising juggernaut on pace to do more than $80 billion in revenue this year at 80+% gross margins, which has only become more critical for advertisers during the pandemic. Because it’s hated, you get all of'

In [20]:
all_feeds = feeds.get_feeds()

for feed_url in all_feeds:
    news_feed = feedparser.parse(feed_url)
    
    for entry in news_feed.entries:
        print(entry['description'] + "~")

Welcome to new subscribers. The Generalist helps you master the future by analyzing the most significant tech and financial market trends. Thank you for being here. There are now over 19,000 of us! Hey friends, The IPO frenzy reached its zenith with DoorDash and Airbnb making their public debuts this week. Both got off to an explosive start, with DoorDash finishing the week at a $55.5 billion valuation and Airbnb landing around $83 billion.~
Affirm in 1 minute PayPal co-founder Max Levchin is back. Affirm, the &#8220;buy now, pay later&#8221; solution led by Levchin is slated to list on the Nasdaq before the year is out. The wisdom of the company&#8217;s mooted $10 billion valuation may depend on the perspective of the investor in question. Is Affirm a lender, selling a commoditized product? Or is it a payments network, capitalizing on changing consumer preferences to steal share from big banks? For Levchin, Affirm&#8217;s disruption of traditional finance is not just a commercial oppo

What it means for the consumerization of enterprise technology, and why Dropbox is next in line.~
The best partnerships can win in the long run, but the bar has been raised.~
Five reasons for why consumer doesn't need a platform shift to thrive in the 2020s.~
It's Apple and Zoom's to lose, and Amazon, Facebook, Google, Microsoft, NVIDIA, or others' to gain.~
Technology's role in democracy and the paths ahead.~
Offer something for free, at a lower price, or make it much more convenient to use.~
One of the most powerful trends in the U.S. economy has only just begun~
Analyzing a Series A Investment against the 10 Factors Framework~
All Subscriptions Are Not Created Equal~
It's still early days for the increasingly dominant business model of the Internet~
Accessibility and personalization are key ways that technology can help solve it~
Decarbonization is the next big thing.~
What will happen to consumer behavior post-pandemic?~
On investment decision-making and writing.~
Could now be the 

On invoices, collecting cash & the promise of smart contracts~
On skepticism, Neo-luddites, progress and what _should_ we be building?~
On marketing, fan engagement, storytelling and our attention spans~
On the growing list of companies analysing our phone calls~
Or the business case for consumer cooking brands in 2019~
On storytelling, honest marketing, expertise and taking customers on a journey~
Fake news, filter bubbles and where to next for critical thinking~
A (potentially) new way to think about product prioritisation, roadmaps & outcomes~
On relationships, counting friends, being busy, & tech-mindfulness~
Teams v Slack, Bundling/Unbundling, Microsoft's future & "deep" products~
The role of radio in a streaming world, music discovery, algorithms and the power of context~
Something I&#8217;ve been pondering lately is how often it seems fast-growing technology companies run into an intriguing dilemma: as the company scales, their biggest strength ultimately becomes their biggest w

How to use ML to diagnose diseases faster~
My conversation with Ryan Delk, CEO of Primer~
What changes to education during COVID are lasting trends and what are quarantine only innovations?~
What trends were discussed at this year's virtual version of education's leading conference~
Corporations are getting involved in student curriculum... could this be the future of higher education?~
America's answer to reopening schools: going outside~
The most important thing we can do is show students the linear path to their first job~
Straight from the mouths of some of the most senior women in edtech, here is what needs to be created today~
How will students come back from the damage the pandemic caused on their learning?~
The future of SEL software~
Jul 24, 2020 8:03 AM~
Jul 17, 2020 8:48 AM~
Welcome to Megan&#8217;s EdTech Newsletter by me, Megan O'Connor. EIR @kaplantestprep &#9999;&#65039;, Co-Founder and CEO @HiClarkTutors, @Human_Ventures and @pencilsofpromise alumni, @inc columnist Sign

7-min read This is the second in a series of posts about one of the most accretive paradigm shifts in our economy since globalization and digitization — dispersion. The market added half a trillion in value in the past two weeks: AT&#38;T busted a baller move to a rundle, the FTC filed suit against Facebook, [&#8230;]~
5-min read The pandemic’s most enduring feature will be as an accelerant of existing trends. The trend that encapsulates the greatest reshuffling of stakeholder value in recent history is … the Great Dispersion. Similar to prior macro trends like globalization and digitization, it offers enormous opportunity, but also real threats. In 1997, I was asked [&#8230;]~
2-min read This is not the first holiday season shrouded in tragedy. When Americans sat down to a peaceful Thanksgiving dinner in 1940, my mother went to sleep in a London Tube station while the Luftwaffe terrorized her city. I’m thankful for those who came before us, finding their way through plagues, wars, and

...

&#129321; Community Wisdom - Issue 13~
Ten strategies and 49 examples of successful buzz making~
&#129321; Community Wisdom - Issue 12~
&#128075; Hello, I&#8217;m Lenny, and welcome to a &#128274; subscriber-only edition &#128274; of my weekly newsletter. Each week I tackle reader questions about product, growth, working with humans, and anything else that&#8217;s stressing you out at the office. Send me your questions~
PMing during COVID, keeping track of everyone's projects, working with PR, ideal team ratios, startup job titles, fireside chat with Shreyas, and much more~
By Pete Kazanjy~
Bootstrapping marketplace supply, leveling up as a first-year PM, tools for tracking OKRs, assessing risk, fostering a culture of feedback, and much more~
&#128075; Hello, I&#8217;m Lenny, and welcome to a &#128274; subscriber-only edition &#128274; of my weekly newsletter. Each week I tackle reader questions about product, growth, working with humans, and anything else that&#8217;s stressing you ou

<p><em>By </em><a href="https://medium.com/@richard.d.mathera"><em>Richard Mathera</em></a></p><p>The election in November is important. Generationally important. And somehow get out the vote (GOTV) efforts are missing one of the most critical aspects of this year’s election.</p><p>Hoards of people are working tirelessly to get people registered to vote and voting. Organizations are agonizing over the most impactful way to write a “remember to vote” letter. People are making GOTV phone calls and having to armor up to face rejection after rejection. Nonprofits are fantasizing about what inspirational SMS messages will motivate a potential voter to register.</p><p>But they might not be asking one critical question that could sneakily be the difference between someone voting and not: <strong>do they have a printer?</strong></p><p>In behavioral science, we analyze the environments in which people act. And in the world of COVID-19, the environment leading up to this year’s election is uniqu

In 2019 <a href="http://blog.eladgil.com/2019/06/industry-towns-where-you-start-company.html">I wrote about how "industry towns" emerge in every market</a>. These clusters of people, ideas, capital, service providers, and companies tend to have strong network effects that support startup formation and success in a given industry.&nbsp;<div><br /></div><div><div class="separator" style="clear: both; text-align: left;"><a href="https://lh3.googleusercontent.com/-_UzwaCV7_5c/X5L_1_5YitI/AAAAAAAAHsU/W9PU5MhI7d4m8ZPMsHgu75FXa1do6FE9ACLcBGAsYHQ/image.png" style="margin-left: 1em; margin-right: 1em;"><img alt="" height="229" src="https://lh3.googleusercontent.com/-_UzwaCV7_5c/X5L_1_5YitI/AAAAAAAAHsU/W9PU5MhI7d4m8ZPMsHgu75FXa1do6FE9ACLcBGAsYHQ/w400-h229/image.png" width="400" /></a></div><br /></div><div><br /></div><div>For example, Silicon Valley, London, Beijing are global tech centers. New York, London, HK, Shanghai etc. are global finance centers. Hollywood, Lagos, Bombay are global movie

<p>Guest Post by Bangaly Kaba (EIR @ Reforge, Former VP Growth @ Instacart, Instagram) &#160; &#160; &#160; &#160; &#160; &#160; The following was written by Bangaly with contributions by other Reforge EIR&#8217;s Elena Verna (Miro, MongoDB, SurveyMonkey) and Fareed Mosavat (Slack, Instacart, Zynga). Reforge is a community for those leading and growing tech companies. To [&#8230;]</p>
<p>The post <a href="https://andrewchen.co/the-adjacent-user-theory/" rel="nofollow">The Adjacent User Theory</a> is also published on <a href="https://andrewchen.co" rel="nofollow">andrewchen</a>.</p>~
<p>Dear readers, I&#8217;ve only been writing sporadically recently on here &#8212; mostly because I&#8217;m working on a new book project (more to come soon on that!!) which has been taking all my time. In the interim, I&#8217;ve stayed pretty active on Twitter, writing tweetstorms that sometimes turn themselves into essays on here. For a quick [&#8230;]</p>
<p>The post <a href="https://andrewchen.co/top-

What are cancer care navigators and what issues do they face?~
COVID tailwinds are changing the role of the pharmacy and pharmacist~
IT'S THE FINAL COUNTDOWNNNNNNN~
Hello again! Just a reminder that the Slack applications are due this Friday 12/4 for the Out-Of-Pocket slack channel. You can fill out the application here I&#8217;d love to see more applications from women and people who are part of underrepresented groups~
How does a pharmacy actually work?~
Video games should be our models for engagement~
Hello! It&#8217;s been about 7 months since I started the Out-Of-Pocket paid slack channel. I&#8217;ve been very, very slowly and deliberately choosing when to add folks and how to actually structure the channel so that it stays engaging. Communities fail when they&#8217;re not thoughtfully developed, so I&#8217;ve been thinking about how the Out-Of-Pocket community can grow while keeping the same level of engagement and quality of conversation.~
Triple. Tax. Advantage.~
Tis the season